# Resolve did:btc1 from local Bitcoin node in regtest

In [1]:
import sys
import os

notebooks_path = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the Notebooks directory to the sys.path
sys.path.append(notebooks_path)

In [2]:
did_to_resolve = "did:btc1:regtest:k1q00gn89qec0t6j0rpavgfgrphh7tmu9p0yz5zpnwwref5n85a28kvnu9laf"


sidecar_data = {
  "did": "did:btc1:regtest:k1q00gn89qec0t6j0rpavgfgrphh7tmu9p0yz5zpnwwref5n85a28kvnu9laf",
  "signalsMetadata": {
    "f6ca3433fe6b015fabaf41412aa6857b7466969078a464fcd0a9605c0d38996b": {
      "updatePayload": {
        "@context": [
          "https://w3id.org/security/v2",
          "https://w3id.org/zcap/v1",
          "https://w3id.org/json-ld-patch/v1"
        ],
        "patch": [
          {
            "op": "add",
            "path": "/service/3",
            "value": {
              "id": "#linked-domain",
              "type": "LinkedDomains",
              "serviceEndpoint": "https://contact-me.com"
            }
          }
        ],
        "sourceHash": "5XcoqeTqmXAYu89hKCNcoWE2R5wQjqze8QrG9HtMmAdC",
        "targetHash": "J9igy4rwejytESdSvqEvFpa8ZvaT8tVwBodyjDrr1KiQ",
        "targetVersionId": 2,
        "proof": {
          "type": "DataIntegrityProof",
          "cryptosuite": "schnorr-secp256k1-jcs-2025",
          "verificationMethod": "did:btc1:regtest:k1q00gn89qec0t6j0rpavgfgrphh7tmu9p0yz5zpnwwref5n85a28kvnu9laf#initialKey",
          "proofPurpose": "capabilityInvocation",
          "@context": [
            "https://w3id.org/security/v2",
            "https://w3id.org/zcap/v1",
            "https://w3id.org/json-ld-patch/v1"
          ],
          "proofValue": "z5pz3vvVjHp1Eaip4AwTeurhKPpozkQxD9xpShcZ8DQ8Ez28VyZyaiUQjtb5W7ugmsJ8BzeomYNpJQfXA4j7hMfaX"
        }
      }
    }
  }
}

In [3]:

resolutionOptions = {"sidecarData": sidecar_data}

## Parse did:btc1 identifier

In [4]:
import re
match = re.search(r"^did:btc1:(?:(\d+):)?(?:(mainnet|signet|testnet|regtest):)?((k1|x1)[023456789acdefghjklmnpqrstuvwxyz]*)$", did_to_resolve)
groups = match.groups()
groups

(None,
 'regtest',
 'k1q00gn89qec0t6j0rpavgfgrphh7tmu9p0yz5zpnwwref5n85a28kvnu9laf',
 'k1')

In [5]:
identifierComponents = {}

identifierComponents["version"] = groups[0] or 1
identifierComponents["network"] = groups[1] or "mainnet"




In [6]:
from libbtc1.bech32 import decode_bech32_identifier

bech32_encoding = groups[2]
hrp, genesisBytes = decode_bech32_identifier(bech32_encoding)

In [7]:
identifierComponents["hrp"] = hrp
identifierComponents["genesisBytes"] = genesisBytes

In [8]:
identifierComponents

{'version': 1,
 'network': 'regtest',
 'hrp': 'k',
 'genesisBytes': b'\x03\xde\x89\x9c\xa0\xce\x1e\xbdI\xe3\x0fX\x84\xa0a\xbd\xfc\xbd\xf0\xa1y\x05A\x06np\xf2\x9aL\xf4\xea\x8ff'}

## Resolve Initial Document

This algorithm specifies how to resolve an initial DID document and validate
it against the `identifier` for a specific **did:btc1**. The algorithm takes as
inputs a **did:btc1** `identifier`, `identifierComponents` object and a
`resolutionsOptions` object. This algorithm returns a valid `initialDocument`
for that identifier.

In [9]:
print(identifierComponents["hrp"])
if identifierComponents["hrp"] == "k":
    print("Deterministically Generate Initial DID Document")

k
Deterministically Generate Initial DID Document


In [10]:
from libbtc1.did import resolve_deterministic

initial_document = resolve_deterministic(did_to_resolve, identifierComponents)

In [11]:
import json
print(json.dumps(initial_document, indent=2))

{
  "id": "did:btc1:regtest:k1q00gn89qec0t6j0rpavgfgrphh7tmu9p0yz5zpnwwref5n85a28kvnu9laf",
  "@context": [
    "https://www.w3.org/ns/did/v1",
    "https://did-btc1/TBD/context"
  ],
  "verificationMethod": [
    {
      "id": "#initialKey",
      "type": "Multikey",
      "controller": "did:btc1:regtest:k1q00gn89qec0t6j0rpavgfgrphh7tmu9p0yz5zpnwwref5n85a28kvnu9laf",
      "publicKeyMultibase": "z66Q1s4kPncjiPQrssyHkcbTFdHC6vi3nf3B1Hsxg8qFwuoT"
    }
  ],
  "authentication": [
    "#initialKey"
  ],
  "assertionMethod": [
    "#initialKey"
  ],
  "capabilityInvocation": [
    "#initialKey"
  ],
  "capabilityDelegation": [
    "#initialKey"
  ],
  "service": [
    {
      "id": "#initial_p2pkh",
      "type": "SingletonBeacon",
      "serviceEndpoint": "bitcoin:n3UVZvtnA64AGQBaMcnM9Bohts9tsbudVm"
    },
    {
      "id": "#initial_p2wpkh",
      "type": "SingletonBeacon",
      "serviceEndpoint": "bitcoin:bcrt1q7rd6q9kf9tl4r5tv8sc9sm046yhry8g94crxqc"
    },
    {
      "id": "#initial_

## Resolve Target Document

This algorithm resolves a DID document from an initial document by walking the
Bitcoin blockchain to identify Beacon Signals that announce DID Update Payloads
applicable to the **did:btc1** identifier being resolved. The algorithm takes
in an `initialDocument` and a set of `resolutionOptions`. The algorithm returns
a valid `targetDocument` or throws an error.

1. If `resolutionOptions.versionId` is not null, set `targetVersionId` to
   `resolutionOptions.versionId`.
1. Else if `resolutionOptions.versionTime` is not null, set `targetTime` to
   `resolutionOptions.versionTime`.
1. Set `targetBlockheight` to the result of passing `targetTime` to the algorithm
   [Determine Target Blockheight].
1. Set `sidecarData` to `resolutionOptions.sidecarData`.
1. Set `currentVersionId` to 1.
1. If `currentVersionId` equals `targetVersionId` return `initialDocument`.
1. Set `updateHashHistory` to an empty array.
1. Set `contemporaryBlockheight` to 0.
1. Set `contemporaryDIDDocument` to the `initialDocument`.
1. Set `targetDocument` to the result of calling the [Traverse Blockchain History]
   algorithm passing in `contemporaryDIDDocument`, `contemporaryBlockheight`,
   `currentVersionId`, `targetVersionId`, `targetBlockheight`, `updateHashHistory`, 
   and `sidecarData`.
1. Return `targetDocument`.

### 1. If resolutionOptions.versionId is not null, set targetVersionId to resolutionOptions.versionId.
### 2. Else if `resolutionOptions.versionTime` is not null, set `targetTime` to `resolutionOptions.versionTime`.

In [12]:
versionId = resolutionOptions.get("versionId")
versionTime = resolutionOptions.get("versionTime")
targetTime = None
targetVersionId = None
if versionId:
    targetVersionId = versionId
elif versionTime:
    targetTime = versionTime

### 3. Set targetBlockheight to the result of passing targetTime to the algorithm [Determine Target Blockheight].

## Determine Target Blockheight

This algorithm takes in an OPTIONAL Unix `targetTime` and returns a Bitcoin
`blockheight`.

In [13]:
required_confirmations = 6
if targetTime:
    raise "NotImplemented"
else:
    print("find latest block with at least x confimations")



find latest block with at least x confimations


In [14]:
from bitcoinrpc import BitcoinRPC
rpc = BitcoinRPC.from_config("http://localhost:18443", ("polaruser", "polarpass"))
await rpc.getconnectioncount()

0

In [15]:
best_blockhash = await rpc.acall("getbestblockhash", {})
bestblock = await rpc.acall("getblock", {"blockhash": best_blockhash})

In [16]:
confirmations = bestblock["confirmations"]
bestblock_height = bestblock["height"]

In [17]:
targetblockheight = bestblock_height + (confirmations - required_confirmations)

In [18]:
targetblockheight

141

### 4. Set `sidecarData` to `resolutionOptions.sidecarData`.

In [19]:
sidecarData = resolutionOptions["sidecarData"]

### 5. Set `currentVersionId` to 1.


In [20]:
currentVersionId = 1

### 6. If `currentVersionId` equals `targetVersionId` return `initialDocument`.

In [21]:
if currentVersionId == targetVersionId:
    print("returning", initial_document)

### 7. Set `updateHashHistory` to an empty array.

In [22]:
updateHashHistory = []

### 8. Set `contemporaryBlockheight` to 1.

In [23]:
contemporaryBlockheight = 0

### 9. Set contemporaryDIDDocument to initialDocument

In [24]:
import copy
contemporaryDidDocument = copy.deepcopy(initial_document)

### 10. Set `targetDocument` to the result of calling the [Traverse Blockchain History] algorithm passing in `contemporaryDIDDocument`, `contemporaryBlockheight`,  `currentVersionId`, `targetVersionId`, `targetBlockheight`, `updateHashHistory`, and `sidecarData`.

# Traverse Blockchain History

### 1. Set `contemporaryHash` to the result of passing `contemporaryDIDDocument` into the [JSON Canonicalization and Hash] algorithm.

In [25]:
import jcs
from buidl.helper import sha256
canonicalDocument = jcs.canonicalize(contemporaryDidDocument)

contemporaryHash = sha256(canonicalDocument)

print(contemporaryHash)

b'CF\xa8\xe3a\x8a\x02\x7f\x16-\r\xe6J\xaez\x05L8o\xeb/T\x8d\xdck\xae1\x17W\xc6\xf6w'


### 2. Find all `beacons` in `contemporaryDIDDocument`: All `service` in `contemporaryDIDDocument.services` where `service.type` equals one of `SingletonBeacon`, `CIDAggregateBeacon` and `SMTAggregateBeacon` Beacon.

In [26]:
beacons = []

for service in contemporaryDidDocument["service"]:
    serviceType = service["type"]
    if serviceType == "SingletonBeacon" or serviceType == "CIDAggregateBeacon" or serviceType == "SMTAggregateBeacon":
        beacons.append(service)


print(beacons)

[{'id': '#initial_p2pkh', 'type': 'SingletonBeacon', 'serviceEndpoint': 'bitcoin:n3UVZvtnA64AGQBaMcnM9Bohts9tsbudVm'}, {'id': '#initial_p2wpkh', 'type': 'SingletonBeacon', 'serviceEndpoint': 'bitcoin:bcrt1q7rd6q9kf9tl4r5tv8sc9sm046yhry8g94crxqc'}, {'id': '#initial_p2tr', 'type': 'SingletonBeacon', 'serviceEndpoint': 'bitcoin:bcrt1pn5pc3em9vw9vw3l3g28tg2nfa2h09efrlq3x3cgmsz6qw2lxuv2sdypydk'}]


### 3. For each `beacon` in `beacons` convert the `beacon.serviceEndpoint` to a Bitcoin address following **[BIP21](https://github.com/bitcoin/bips/blob/master/bip-0021.mediawiki)**. Set `beacon.address` to the Bitcoin address.

In [27]:
for beacon in beacons:
    serviceEndpoint = beacon["serviceEndpoint"]
    beacon["address"] = serviceEndpoint[8:]

print(beacons)

[{'id': '#initial_p2pkh', 'type': 'SingletonBeacon', 'serviceEndpoint': 'bitcoin:n3UVZvtnA64AGQBaMcnM9Bohts9tsbudVm', 'address': 'n3UVZvtnA64AGQBaMcnM9Bohts9tsbudVm'}, {'id': '#initial_p2wpkh', 'type': 'SingletonBeacon', 'serviceEndpoint': 'bitcoin:bcrt1q7rd6q9kf9tl4r5tv8sc9sm046yhry8g94crxqc', 'address': 'bcrt1q7rd6q9kf9tl4r5tv8sc9sm046yhry8g94crxqc'}, {'id': '#initial_p2tr', 'type': 'SingletonBeacon', 'serviceEndpoint': 'bitcoin:bcrt1pn5pc3em9vw9vw3l3g28tg2nfa2h09efrlq3x3cgmsz6qw2lxuv2sdypydk', 'address': 'bcrt1pn5pc3em9vw9vw3l3g28tg2nfa2h09efrlq3x3cgmsz6qw2lxuv2sdypydk'}]


### 4. Set `nextSignals` to the result of calling algorithm Find Next Signals passing in `contemporaryBlockheight` and `beacons`.


# Find Next Signals

This algorithm takes in a `contemporaryBlockheight` and a set of `beacons` and
finds the next Bitcoin block containing Beacon Signals from one or more of the
`beacons`.

In [32]:
from buidl.tx import Tx, TxIn

async def find_next_signals(contemporaryBlockheight, targetBlockheight, beacons):
    # 1. Get Bitcoin block at contemporaryBlockheight
    blockhash = await rpc.acall("getblockhash", {"height": contemporaryBlockheight})
    block = await rpc.acall("getblock", {"blockhash": blockhash})
    
    # 2. Set `beaconSignals` to an empty array.
    beaconSignals = {
        "signals": [],
        "blockheight": contemporaryBlockheight
    }
    
    # 3.For each `txid` in `block.tx`: 
    # check to see if any transaction inputs are spends from one of the Beacon addresses.
    # If they are, create a `signal` object containing the following fields and push
    # `signal` to `beaconSignals`:
    for txid in block["tx"]:

        #  Skip coinbase
        
        # 1. Fetch transaction for txid
        tx_hex = await rpc.acall("getrawtransaction", {"txid": txid})
        tx = Tx.parse_hex(tx_hex)
        
        
        # For each tx input in transaction
        for tx_in in tx.tx_ins:    
            # If tx_in is NOT a coinbase transaction
            prev_txid = tx_in.prev_tx.hex()
            if prev_txid != "0000000000000000000000000000000000000000000000000000000000000000":
                # Fetch previous tx
                prev_tx_hex = await rpc.acall("getrawtransaction", {"txid": prev_txid})
                prev_tx = Tx.parse_hex(prev_tx_hex)
                # Get spent output
                spent_tx_output = prev_tx.tx_outs[tx_in.prev_index]
                print("Bitcoin block at height : ", contemporaryBlockheight)
                print(spent_tx_output.script_pubkey.address(network="regtest"))
                # Get address for the spent output 
                spent_tx_output_address = spent_tx_output.script_pubkey.address(network="regtest")
                from_beacon = None
                # Check if address is a beacon address
                for beacon in beacons:
                    if beacon["address"] == spent_tx_output_address:
                        signal = {
                            "beaconId": beacon["id"],
                            "beaconType": beacon["type"],
                            "tx": tx
                        }
                        beaconSignals["signals"].append(signal)
                        print("Found Beacon Signal", tx)
                        break
                # If found beacon, no need to check other inputs
                # Is that true?
                if from_beacon:
                    break

    if contemporaryBlockheight == targetBlockheight:
        return beaconSignals

    
    if len(beaconSignals["signals"]) == 0:
        next_blockheight = contemporaryBlockheight + 1
        beaconSignals = await find_next_signals(next_blockheight, targetBlockheight, beacons)

    return beaconSignals
                    


In [33]:

nextSignals = await find_next_signals(1, targetblockheight, beacons)

Bitcoin block at height :  102
bcrt1quszy492dyjwq4asuryr8f72pndteuyea6ea589
Bitcoin block at height :  109
bcrt1qtux996xmz6xgv0kp5d9t4xmxnmgdw2rdkz70nm
Bitcoin block at height :  115
bcrt1q06m9yn2kxgxg2mara55667958d0a6s2ksdc3dd
Bitcoin block at height :  123
bcrt1qdj6g6jcjyt9u9akcuwtgm6kxl9mss3068cd6jh
Bitcoin block at height :  129
bcrt1ql7q9485kxz4330hxtknqkkq6ym6jtc8snpyg5x
Bitcoin block at height :  129
bcrt1q7w7f5yqqdem9fvp6dm9d2pnmevp556fwkm4ehf
Bitcoin block at height :  129
bcrt1q3gfs3vk7jz6uj4luqmq74lswawf2wg96vpplgl
Bitcoin block at height :  135
bcrt1qjy7hmknygpgq4343zk7vknw2y4fp0wsdnhqyzm
Bitcoin block at height :  135
bcrt1q7rd6q9kf9tl4r5tv8sc9sm046yhry8g94crxqc
Found Beacon Signal 
tx: f6ca3433fe6b015fabaf41412aa6857b7466969078a464fcd0a9605c0d38996b
version: 1
locktime: 0
tx_ins:
3c2dfc65402e52f3e449f163349748c695eb17c4db31fdde9b0727cf8137740a:1
tx_outs:
0:OP_RETURN fa4f9cc77516ac5c1a201f87fcdc18017b80ed0d8cee81e2a7f21037cff4330a 
19999650:OP_0 f0dba016c92aff51d16c3c30586

### 5. Set `signals` to `nextSignals.signals`.


In [34]:
signals = nextSignals["signals"]

### 6. Set `updates` to the result of calling algorithm [Process Beacon Signals] passing in `signals` and `sidecarData`.

# Process Beacon Signals

This algorithm takes in an array of struct `beaconSignals` and attempts
to process these signals according the type of the Beacon they were produced by.
Each `beaconSignal` struct contains the properties `beaconId`, `beaconType`, and
a `tx`. Additionally, this algorithm takes in `sidecarData` passed into the
resolver through the `resolutionOptions`. If `sidecarData` is present it is used
to process the Beacon Signals.

In [35]:
print(signals[0]["tx"])
print(sidecarData)


tx: f6ca3433fe6b015fabaf41412aa6857b7466969078a464fcd0a9605c0d38996b
version: 1
locktime: 0
tx_ins:
3c2dfc65402e52f3e449f163349748c695eb17c4db31fdde9b0727cf8137740a:1
tx_outs:
0:OP_RETURN fa4f9cc77516ac5c1a201f87fcdc18017b80ed0d8cee81e2a7f21037cff4330a 
19999650:OP_0 f0dba016c92aff51d16c3c30586df5d12e321d05 

{'did': 'did:btc1:regtest:k1q00gn89qec0t6j0rpavgfgrphh7tmu9p0yz5zpnwwref5n85a28kvnu9laf', 'signalsMetadata': {'f6ca3433fe6b015fabaf41412aa6857b7466969078a464fcd0a9605c0d38996b': {'updatePayload': {'@context': ['https://w3id.org/security/v2', 'https://w3id.org/zcap/v1', 'https://w3id.org/json-ld-patch/v1'], 'patch': [{'op': 'add', 'path': '/service/3', 'value': {'id': '#linked-domain', 'type': 'LinkedDomains', 'serviceEndpoint': 'https://contact-me.com'}}], 'sourceHash': '5XcoqeTqmXAYu89hKCNcoWE2R5wQjqze8QrG9HtMmAdC', 'targetHash': 'J9igy4rwejytESdSvqEvFpa8ZvaT8tVwBodyjDrr1KiQ', 'targetVersionId': 2, 'proof': {'type': 'DataIntegrityProof', 'cryptosuite': 'schnorr-secp256k1-jcs-202

In [51]:
def process_beacon_signals(beaconSignals, sidecarData):
    # 1. Set `updates` to an empty array.
    updates = []
    # 2. For `beaconSignal` in `beaconSignals`:
    for beaconSignal in beaconSignals:
        beaconType = beaconSignal["beaconType"]
        signalTx = beaconSignal["tx"]
        signalId = signalTx.id()
        signalSidecarData = sidecarData["signalsMetadata"].get(signalId)
        didUpdatePayload = None
        if beaconType == "SingletonBeacon":
            didUpdatePayload = process_singleton_beacon_signal(signalTx, signalSidecarData)
        # Handle other cases

        if didUpdatePayload:
            updates.append(didUpdatePayload)
    return updates

# Process Singleton Beacon Signal

This algorithm is called by the [Process Beacon Signals] algorithm as part of the
[Read] operation. It takes as inputs a Bitcoin transaction, `tx`, representing a ::Beacon Signal::
and a optional object, `signalSidecarData`, containing any sidecar data provided to the 
resolver for the ::Beacon Signal:: identified by the Bitcoin transaction identifier.

The algorithm returns the ::DID Update payload:: announced by the ::Beacon Signal:: or throws
an error.

1. Initialize a `txOut` variable to the 0th transaction output of the `tx`.
1. Set `didUpdatePayload` to null.
1. Check `txOut` is of the format `[OP_RETURN, OP_PUSH32, <32bytes>]`, if not,
   then return `didUpdatePayload`. The Bitcoin transaction is not a ::Beacon Signal::.
1. Set `hashBytes` to the 32 bytes in the `txOut`.
1. If `signalSidecarData`:
   1. Set `didUpdatePayload` to `signalSidecarData.updatePayload`
   1. Set `updateHashBytes` to the result of passing `didUpdatePayload` to the 
      [JSON Canonicalization and Hash] algorithm.
   1. If `updateHashBytes` does not equal `hashBytes`, MUST throw an `invalidSidecarData` error.
   1. Return `didUpdatePayload`
1. Else:
   1. Set `didUpdatePayload` to the result of passing `hashBytes` into the 
      [Fetch Content from Addressable Storage] algorithm.
   1. If `didUpdatePayload` is null, MUST raise a `latePublishingError`. May identify Beacon Signal
      to resolver and request additional ::Sidecar data:: be provided.
1. Return `didUpdatePayload`.

In [52]:
from ipfs_cid import cid_sha256_wrap_digest


def process_singleton_beacon_signal(tx, signalSidecarData):
    txOut = tx.tx_outs[0]
    didUpdatePayload = None
    # Note: No OP_PUSH32. It is implied by the length of the bytes.
    if (txOut.script_pubkey.commands[0] != 106 and len(txOut.script_pubkey.commands[1]) != 32):
        print("Not a beacon signal")
        return didUpdatePayload
    hashBytes = txOut.script_pubkey.commands[1]
    if signalSidecarData:
        didUpdatePayload = signalSidecarData["updatePayload"]
        updateHashBytes = sha256(jcs.canonicalize(didUpdatePayload))
        
        print(updateHashBytes)
        print(hashBytes)
        if updateHashBytes != hashBytes:
            raise Exception("Invalid Sidecar Data")
        return didUpdatePayload
    else:
        payload_cid = cid_sha256_wrap_digest(hashBytes)
        print("TODO: Fetch CID from CAS", payload_cid)
        
        

In [53]:
tx = signals[0]["tx"]
txOut = tx.tx_outs[0]
didUpdatePayload = None
print(txOut.script_pubkey.commands[0] == 106)
process_singleton_beacon_signal(tx, sidecarData["signalsMetadata"][tx.id()])

True
b'\xfaO\x9c\xc7u\x16\xac\\\x1a \x1f\x87\xfc\xdc\x18\x01{\x80\xed\r\x8c\xee\x81\xe2\xa7\xf2\x107\xcf\xf43\n'
b'\xfaO\x9c\xc7u\x16\xac\\\x1a \x1f\x87\xfc\xdc\x18\x01{\x80\xed\r\x8c\xee\x81\xe2\xa7\xf2\x107\xcf\xf43\n'


{'@context': ['https://w3id.org/security/v2',
  'https://w3id.org/zcap/v1',
  'https://w3id.org/json-ld-patch/v1'],
 'patch': [{'op': 'add',
   'path': '/service/3',
   'value': {'id': '#linked-domain',
    'type': 'LinkedDomains',
    'serviceEndpoint': 'https://contact-me.com'}}],
 'sourceHash': '5XcoqeTqmXAYu89hKCNcoWE2R5wQjqze8QrG9HtMmAdC',
 'targetHash': 'J9igy4rwejytESdSvqEvFpa8ZvaT8tVwBodyjDrr1KiQ',
 'targetVersionId': 2,
 'proof': {'type': 'DataIntegrityProof',
  'cryptosuite': 'schnorr-secp256k1-jcs-2025',
  'verificationMethod': 'did:btc1:regtest:k1q00gn89qec0t6j0rpavgfgrphh7tmu9p0yz5zpnwwref5n85a28kvnu9laf#initialKey',
  'proofPurpose': 'capabilityInvocation',
  '@context': ['https://w3id.org/security/v2',
   'https://w3id.org/zcap/v1',
   'https://w3id.org/json-ld-patch/v1'],
  'proofValue': 'z5pz3vvVjHp1Eaip4AwTeurhKPpozkQxD9xpShcZ8DQ8Ez28VyZyaiUQjtb5W7ugmsJ8BzeomYNpJQfXA4j7hMfaX'}}

In [54]:
updates = process_beacon_signals(signals, sidecarData)

b'\xfaO\x9c\xc7u\x16\xac\\\x1a \x1f\x87\xfc\xdc\x18\x01{\x80\xed\r\x8c\xee\x81\xe2\xa7\xf2\x107\xcf\xf43\n'
b'\xfaO\x9c\xc7u\x16\xac\\\x1a \x1f\x87\xfc\xdc\x18\x01{\x80\xed\r\x8c\xee\x81\xe2\xa7\xf2\x107\xcf\xf43\n'


In [55]:
updates

[{'@context': ['https://w3id.org/security/v2',
   'https://w3id.org/zcap/v1',
   'https://w3id.org/json-ld-patch/v1'],
  'patch': [{'op': 'add',
    'path': '/service/3',
    'value': {'id': '#linked-domain',
     'type': 'LinkedDomains',
     'serviceEndpoint': 'https://contact-me.com'}}],
  'sourceHash': '5XcoqeTqmXAYu89hKCNcoWE2R5wQjqze8QrG9HtMmAdC',
  'targetHash': 'J9igy4rwejytESdSvqEvFpa8ZvaT8tVwBodyjDrr1KiQ',
  'targetVersionId': 2,
  'proof': {'type': 'DataIntegrityProof',
   'cryptosuite': 'schnorr-secp256k1-jcs-2025',
   'verificationMethod': 'did:btc1:regtest:k1q00gn89qec0t6j0rpavgfgrphh7tmu9p0yz5zpnwwref5n85a28kvnu9laf#initialKey',
   'proofPurpose': 'capabilityInvocation',
   '@context': ['https://w3id.org/security/v2',
    'https://w3id.org/zcap/v1',
    'https://w3id.org/json-ld-patch/v1'],
   'proofValue': 'z5pz3vvVjHp1Eaip4AwTeurhKPpozkQxD9xpShcZ8DQ8Ez28VyZyaiUQjtb5W7ugmsJ8BzeomYNpJQfXA4j7hMfaX'}}]

### 7. Set `orderedUpdates` to the list of `updates` ordered by the `targetVersionId` property.

In [58]:
updates.sort(key=lambda update: update["targetVersionId"])

### 8. For `update` in `orderedUpdates`:
1. If `update.targetVersionId` is less than or equal to `currentVersionId`,
   run Algorithm [Confirm Duplicate Update] passing in `update`,
   `documentHistory`, and `contemporaryHash`.
1. If `update.targetVersionId` equals `currentVersionId + 1`:
    1.  Check that `update.sourceHash` equals `contemporaryHash`, else MUST
        raise LatePublishing error.
    1.  Set `contemporaryDIDDocument` to the result of calling [Apply DID Update]
        algorithm passing in `contemporaryDIDDocument`, `update`.
    1.  Increment `currentVersionId`
    1.  If `currentVersionId` equals `targetVersionId` return
        `contemporaryDIDDocument`.
    1.  Set `updateHash` to the sha256 hash of the `update`.
    1.  Push `updateHash` onto `updateHashHistory`.
    1.  Set `contemporaryHash` to the SHA256 hash of the
        `contemporaryDIDDocument`.
1.  If `update.targetVersionId` is greater than `currentVersionId + 1`, MUST
    throw a LatePublishing error.

In [66]:
for update in updates:
    targetVersionId = update["targetVersionId"]
    if targetVersionId == currentVersionId:
        confirm_duplicate_update(update, documentHistory, contemporaryHash)
    if targetVersionId == currentVersionId + 1:
        assert(update["sourceHash"] == contemporaryHash, "Late Publishing")
        contemporaryDidDocument = apply_did_update(contemporaryDidDocument, update)
        currentVersionId +=1
        if currentVersionId == targetVersionId:
            print("Found document for target version", contemporaryDidDocument)
        updateHash = sha256(jcs.canonicalize(update))
        contemporaryHash = sha256(jcs.canonicalize(contemporaryDidDocument))
    if targetVersionId > currentVersionId + 1:
        raise Exception("Late publishing")
        

<>:6: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:6: SyntaxWarning: assertion is always true, perhaps remove parentheses?
/tmp/ipykernel_13616/3163534492.py:6: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(update["sourceHash"] == contemporaryHash, "Late Publishing")


NameError: name 'apply_did_update' is not defined

# Confirm Duplicate Update

This algorithm takes in a DID Update Payload and verifies that the update is a
duplicate against the hash history of previously applied updates.
The algorithm takes in an `update` and an array of hashes, `updateHashHistory`.
It throws an error if the `update` is not a duplicate, otherwise it returns.
TODO: does this algorithm need  `contemporaryHash` passed in?

In [68]:
def confirm_duplicate_update(update, updateHashHistory):

    updateHash = sha256(jcs.canonicalize(update))
    # Note: version starts at 1, index starts at 0
    updateHashIndex = update["sourceVersionId"] - 1
    historicalUpdateHash = updateHashHistory[updateHashIndex]
    if (historicalUpdateHash != updateHash):
        raise Exception("Late Publishing Error")
    return    

# Apply DID Update

This algorithm attempts to apply a DID Update to a DID document, it first
verifies the proof on the update is a valid capabilityInvocation of the root
authority over the DID being resolved. Then it applies the JSON patch
transformation to the DID document, checks the transformed DID document
matches the targetHash specified by the update and validates it is a conformant
DID document before returning it. This algorithm takes inputs
`contemporaryDIDDocument` and an `update`.

In [76]:
for txid in block["tx"]:
    # 1. Fetch transaction for txid
    tx_hex = await rpc.acall("getrawtransaction", {"txid": "f6ca3433fe6b015fabaf41412aa6857b7466969078a464fcd0a9605c0d38996b"})
    tx = Tx.parse_hex(tx_hex)
    for tx_in in tx.tx_ins:    
        # If tx_in is NOT a coinbase transaction
        prev_txid = tx_in.prev_tx.hex()
        if prev_txid != "0000000000000000000000000000000000000000000000000000000000000000":
            prev_tx_hex = await rpc.acall("getrawtransaction", {"txid": prev_txid})
            prev_tx = Tx.parse_hex(prev_tx_hex)
            spent_tx_output = prev_tx.tx_outs[tx_in.prev_index]
            print("Bitcoin block at height : ", contemporaryBlockheight)
            print(spent_tx_output.script_pubkey.address(network="regtest"))

Bitcoin block at height :  1
bcrt1q7rd6q9kf9tl4r5tv8sc9sm046yhry8g94crxqc
Bitcoin block at height :  1
bcrt1q7rd6q9kf9tl4r5tv8sc9sm046yhry8g94crxqc
Bitcoin block at height :  1
bcrt1q7rd6q9kf9tl4r5tv8sc9sm046yhry8g94crxqc
Bitcoin block at height :  1
bcrt1q7rd6q9kf9tl4r5tv8sc9sm046yhry8g94crxqc
